<a href="https://colab.research.google.com/github/rabbitmetrics/langchain-13-min/blob/main/notebooks/langchain-13-min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load environment variables

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

False

In [3]:
# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

'\n\nLarge language models are powerful, deep neural networks trained on large amounts of text data, which can be used to generate text, answer questions, and perform other natural language processing tasks.'

In [4]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [5]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')

Sure, here's an example Python script that trains a simple neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
X = np.random.rand(1000, 10)
y = np.sum(X, axis=1)

# Define the neural network architecture
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)
```

In this script, we first generate simulated data consisting of 1000 samples with 10 features each, where the target variable is the sum of the features. We then define a simple neural network architecture with one hidden layer of 32 units and a linear output layer. We compile the model with mean squared error loss and the Adam optimizer, and then train the model for 100 epochs with a batch siz

In [6]:
# Import prompt and define PromptTemplate

from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [7]:
# Run LLM with PromptTemplate

llm(prompt.format(concept="autoencoder"))

'\nAn autoencoder is a type of artificial neural network used for unsupervised learning. It is designed to learn a compressed representation of the input data by training the network to reconstruct the original input from the compressed representation. Autoencoders can be used for dimensionality reduction, anomaly detection, and generative modeling.'

In [8]:
# Import LLMChain and define chain with language model and prompt as arguments.

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))


An autoencoder is a type of artificial neural network used to learn efficient data representations in an unsupervised manner. It consists of an encoder which compresses the input data into a latent representation, and a decoder which reconstructs the input from the latent representation.


In [9]:
# Define a second prompt 

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
# Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...

An autoencoder is a type of neural network that is used to learn a compressed representation of data. It learns to represent the input data as a reduced set of features, which can then be used for various tasks such as feature extraction, dimensionality reduction, and reconstruction.


An autoencoder is like a magic box. It takes something in, like a photo of your dog, and it makes it smaller. It takes all the details, like the color of your dog’s fur, the shape of its ears, and the size of its collar and makes them into something simpler. 

Think of it like a zipper. You unzip a jacket and all the details of the jacket are gone. Now it’s just a flat piece of material. That’s what an autoencoder does. It takes the details of an image and turns it into something simpler. 

The autoencoder then takes the simplified version of the image and tries to recreate the original image as closely as possible. It takes the simplified version of the i

In [11]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])


In [12]:
# Individual text chunks can be accessed with "page_content"

texts[0].page_content

'An autoencoder is like a magic box. It takes something in, like a photo of your dog, and it makes it'

In [13]:
# Import and instantiate OpenAI embeddings

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="ada")

In [15]:
# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[-0.022758472283970627, 0.020555713759742356, -0.0005732125611561271, 0.03165072143315715, 0.039751197397159745, 0.02937690486046369, -0.01780479926747373, 0.017013024219644693, 0.03053411597855622, 0.021499752595084437, -0.04498909701958205, -0.026676747447559566, 0.029092678254538287, 0.06366687893657778, 0.042837089986782935, -0.0053647871985594885, 0.025357121126081093, 0.01922593602239098, 0.00460092668173581, -0.0018106791782499267, 0.020809487980694173, -0.019510162628316383, 0.08892249149186522, -0.024646553679945025, 0.04117233228943188, 0.0404820658123965, 0.004542558834434305, 0.046126003443283665, -0.01000377982717356, -0.031001060619613368, -0.010176346446432404, 0.011277726639869099, 0.03638107261367581, 0.02160126302852321, 0.018951858969693362, 0.014038790685365508, 0.03705103625896539, 0.020383147140483508, -0.024626250848199225, -0.028483619379195878, 0.014383923923883197, -0.016241550140916338, 0.02334723019021235, 0.006958489641413024, -0.008095398393421032, 0.00704

In [27]:
# Import and initialize Pinecone client

import os
import pinecone
from langchain.vectorstores import Pinecone


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')
)

In [28]:
# Upload vectors to Pinecone

index_name = "langchain-quickstart"
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

TypeError: expected string or bytes-like object

In [ ]:
# Do a simple vector similarity search

query = "What is magical about an autoencoder?"
result = search.similarity_search(query)

print(result)

In [ ]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")